# Chapter 06-05: Sequence-to-Sequence (Seq2Seq)

## 학습 목표
- Seq2Seq 아키텍처(인코더-디코더)의 구조와 작동 원리를 이해한다.
- Context Vector의 개념과 한계를 파악한다.
- 날짜 형식 변환 예제로 Seq2Seq 모델을 직접 구현한다.
- Attention 메커니즘의 필요성을 이해한다.

## 목차
1. [기본 임포트](#1.-기본-임포트)
2. [Seq2Seq 아키텍처](#2.-Seq2Seq-아키텍처)
3. [날짜 변환 데이터 생성](#3.-날짜-변환-데이터-생성)
4. [Encoder-Decoder LSTM 구현](#4.-Encoder-Decoder-LSTM-구현)
5. [학습 및 예측](#5.-학습-및-예측)
6. [Attention의 필요성](#6.-Attention의-필요성)

In [ ]:
# 기본 라이브러리 임포트
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")

# 재현성을 위한 시드 설정
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

## 2. Seq2Seq 아키텍처

### 인코더-디코더 구조

Seq2Seq 모델은 가변 길이의 입력 시퀀스를 가변 길이의 출력 시퀀스로 변환한다.

```
[입력 시퀀스]          [출력 시퀀스]
"2024년 1월 15일"  →  "2024-01-15"
"I love you"       →  "나는 너를 사랑해"
```

**구성 요소:**

1. **인코더(Encoder)**
   - 입력 시퀀스 전체를 읽어 고정 크기의 **Context Vector**($h_T$)로 압축
   - 마지막 타임스텝의 은닉 상태가 Context Vector가 됨

2. **Context Vector**
   - 입력 시퀀스의 의미를 담은 고정 크기 벡터
   - 인코더의 출력을 디코더의 초기 상태로 전달

3. **디코더(Decoder)**
   - Context Vector에서 시작하여 출력 시퀀스를 한 토큰씩 생성
   - 이전 타임스텝의 출력을 다음 타임스텝의 입력으로 사용 (자기회귀)

### Context Vector의 한계

고정 크기 벡터에 모든 입력 정보를 압축하므로:
- 긴 시퀀스에서 **정보 손실** 발생
- 입력 초반부 단어의 정보가 **희석**됨
- 이를 해결하기 위해 **Attention 메커니즘** 도입

In [ ]:
# 날짜 형식 변환 예제 데이터 생성
# 과제: "2024년 1월 15일" → "2024-01-15"

# 한국어 날짜 → ISO 형식 날짜 변환
def generate_date_pairs(n_samples=5000):
    """랜덤 날짜 쌍 생성 (한국어 형식 → ISO 형식)"""
    months_kr = [
        '1월', '2월', '3월', '4월', '5월', '6월',
        '7월', '8월', '9월', '10월', '11월', '12월'
    ]
    days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    pairs = []
    years = list(range(2000, 2026))  # 2000~2025년
    
    for _ in range(n_samples):
        year  = random.choice(years)
        month = random.randint(1, 12)
        day   = random.randint(1, days_per_month[month - 1])
        
        # 입력: 한국어 날짜 형식
        source = f"{year}년 {months_kr[month-1]} {day}일"
        # 출력: ISO 날짜 형식
        target = f"{year}-{month:02d}-{day:02d}"
        
        pairs.append((source, target))
    
    return pairs

# 데이터 생성
date_pairs = generate_date_pairs(5000)

print("생성된 날짜 변환 예시:")
for src, tgt in date_pairs[:8]:
    print(f"  입력: {src:20s}  →  출력: {tgt}")
print(f"\n총 데이터 수: {len(date_pairs):,d}")

# 입력/출력 분리
sources = [p[0] for p in date_pairs]
targets = [p[1] for p in date_pairs]

# 출력 시퀀스에 시작/종료 토큰 추가
# 디코더 학습: <start> 토큰으로 시작
# 디코더 타겟: <end> 토큰으로 끝
targets_in  = ['<start> ' + t for t in targets]  # 디코더 입력
targets_out = [t + ' <end>'    for t in targets]  # 디코더 타겟

print("\n디코더 입력/타겟 예시:")
print(f"  디코더 입력: '{targets_in[0]}'")
print(f"  디코더 타겟: '{targets_out[0]}'")

In [ ]:
# Encoder-Decoder LSTM 구현

# 문자 수준 처리 (날짜 변환은 문자 수준이 적합)
LATENT_DIM = 256  # LSTM 은닉 유닛 수

# 문자 집합 구성 (입력/출력 각각)
all_source_chars = sorted(set(''.join(sources)))
all_target_chars = sorted(set(''.join(targets_in + targets_out)))

print(f"입력 문자 집합 크기: {len(all_source_chars)}")
print(f"  문자들: {all_source_chars}")
print(f"\n출력 문자 집합 크기: {len(all_target_chars)}")
print(f"  문자들: {all_target_chars}")

# 문자 ↔ 인덱스 매핑
src_char2idx = {c: i for i, c in enumerate(all_source_chars)}
tgt_char2idx = {c: i for i, c in enumerate(all_target_chars)}
tgt_idx2char = {i: c for c, i in tgt_char2idx.items()}

NUM_SRC_CHARS = len(all_source_chars)
NUM_TGT_CHARS = len(all_target_chars)

# 최대 시퀀스 길이
max_src_len = max(len(s) for s in sources)
max_tgt_len = max(len(t) for t in targets_in)
print(f"\n최대 입력 길이: {max_src_len}")
print(f"최대 출력 길이: {max_tgt_len}")

# One-Hot 인코딩 데이터 생성
print("\nOne-Hot 인코딩 데이터 생성 중...")
N = len(date_pairs)

encoder_input_data  = np.zeros((N, max_src_len, NUM_SRC_CHARS), dtype='float32')
decoder_input_data  = np.zeros((N, max_tgt_len, NUM_TGT_CHARS), dtype='float32')
decoder_target_data = np.zeros((N, max_tgt_len, NUM_TGT_CHARS), dtype='float32')

for i, (src, tgt_in, tgt_out) in enumerate(zip(sources, targets_in, targets_out)):
    for t, ch in enumerate(src):
        encoder_input_data[i, t, src_char2idx[ch]] = 1.0
    for t, ch in enumerate(tgt_in):
        decoder_input_data[i, t, tgt_char2idx[ch]] = 1.0
    for t, ch in enumerate(tgt_out):
        decoder_target_data[i, t, tgt_char2idx[ch]] = 1.0

print(f"인코더 입력 형태: {encoder_input_data.shape}")
print(f"디코더 입력 형태: {decoder_input_data.shape}")
print(f"디코더 타겟 형태: {decoder_target_data.shape}")

In [ ]:
# Seq2Seq 모델 구성 및 학습

# ── 인코더 ──────────────────────────────────────────────
encoder_inputs = tf.keras.Input(shape=(None, NUM_SRC_CHARS), name='encoder_input')
# return_state=True: 은닉 상태와 셀 상태를 반환 (Context Vector로 사용)
encoder_lstm = tf.keras.layers.LSTM(
    LATENT_DIM,
    return_state=True,  # (output, hidden_state, cell_state) 반환
    name='encoder_lstm'
)
# encoder_outputs: 전체 시퀀스 출력 (사용하지 않음)
# state_h, state_c: 마지막 타임스텝의 은닉/셀 상태 → Context Vector
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # 디코더 초기 상태로 전달

# ── 디코더 ──────────────────────────────────────────────
decoder_inputs = tf.keras.Input(shape=(None, NUM_TGT_CHARS), name='decoder_input')
decoder_lstm = tf.keras.layers.LSTM(
    LATENT_DIM,
    return_sequences=True,  # 모든 타임스텝 출력
    return_state=True,
    name='decoder_lstm'
)
# initial_state=encoder_states: 인코더의 마지막 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# 출력층: 각 타임스텝마다 문자 확률 분포 출력
decoder_dense = tf.keras.layers.Dense(NUM_TGT_CHARS, activation='softmax', name='output')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq 모델 생성
seq2seq_model = tf.keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name='Seq2Seq'
)

seq2seq_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

seq2seq_model.summary()

# 학습 실행
print("\nSeq2Seq 모델 학습 시작...")
history = seq2seq_model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=64,
    epochs=30,
    validation_split=0.2,
    verbose=1
)

In [ ]:
# 예측 함수 구현 (추론 시에는 학습과 다른 방식으로 디코딩)

# ── 추론용 인코더 모델 ───────────────────────────────────
# 입력을 받아 Context Vector(인코더 상태)를 반환
encoder_model = tf.keras.Model(
    encoder_inputs,
    encoder_states,
    name='Encoder_Inference'
)

# ── 추론용 디코더 모델 ───────────────────────────────────
# 이전 상태를 입력으로 받아 다음 토큰과 새로운 상태를 반환
decoder_state_input_h = tf.keras.Input(shape=(LATENT_DIM,), name='dec_state_h')
decoder_state_input_c = tf.keras.Input(shape=(LATENT_DIM,), name='dec_state_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_inputs,
    initial_state=decoder_states_inputs
)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf,
    name='Decoder_Inference'
)

def decode_sequence(input_seq):
    """그리디 디코딩으로 출력 시퀀스 생성"""
    # 1. 인코더로 Context Vector 생성
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    # 2. <start> 토큰으로 디코더 시작
    target_seq = np.zeros((1, 1, NUM_TGT_CHARS))
    target_seq[0, 0, tgt_char2idx['<']] = 1.0  # '<start>' 시작
    
    # 실제로는 '<' 대신 '<start>' 전체를 처리해야 하지만,
    # 이 예제에서는 단순화를 위해 첫 문자만 사용
    
    stop_condition = False
    decoded_sentence = ''
    max_decode_len = 20
    
    while not stop_condition:
        # 3. 디코더로 다음 토큰 예측
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )
        
        # 4. 가장 높은 확률의 토큰 선택 (그리디)
        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        sampled_char = tgt_idx2char[sampled_token_idx]
        
        # 5. 종료 조건 확인
        if sampled_char == '>' or len(decoded_sentence) > max_decode_len:
            stop_condition = True
        else:
            decoded_sentence += sampled_char
        
        # 6. 다음 타임스텝 준비
        target_seq = np.zeros((1, 1, NUM_TGT_CHARS))
        target_seq[0, 0, sampled_token_idx] = 1.0
        states_value = [h, c]  # 상태 업데이트
    
    return decoded_sentence

# 테스트 예측
print("=== 날짜 변환 예측 테스트 ===")
test_indices = random.sample(range(N), 5)
for idx in test_indices:
    input_seq  = encoder_input_data[idx:idx+1]
    predicted  = decode_sequence(input_seq)
    actual     = targets[idx]
    source_txt = sources[idx]
    print(f"  입력:  {source_txt:20s}")
    print(f"  예측:  {predicted}")
    print(f"  실제:  {actual}")
    print(f"  결과:  {'정답' if predicted.strip() == actual else '오답'}")
    print()

## 6. Attention 메커니즘의 필요성

### Context Vector의 병목 문제

Seq2Seq 모델에서 인코더는 입력 시퀀스 전체를 **고정 크기**의 Context Vector로 압축한다.  
이 접근법은 짧은 시퀀스에서는 잘 동작하지만, 긴 시퀀스에서는 한계가 있다.

$$\text{긴 문장} \xrightarrow{\text{인코더}} \underbrace{\vec{c}}_{\text{고정 크기 벡터}} \xrightarrow{\text{디코더}} \text{출력}$$

### Attention의 아이디어

디코더가 출력을 생성할 때 **입력 시퀀스의 모든 타임스텝**을 직접 참조한다.  
각 출력 토큰을 생성할 때마다 **어느 입력에 집중할지(Attention)**를 학습한다.

$$\text{Attention Score}: e_{t,s} = \text{score}(h_t, \bar{h}_s)$$

$$\text{Attention Weight}: \alpha_{t,s} = \frac{\exp(e_{t,s})}{\sum_{s'} \exp(e_{t,s'})}$$

$$\text{Context Vector}: c_t = \sum_s \alpha_{t,s} \bar{h}_s$$

### Transformer로의 발전

Attention 메커니즘은 이후 **Self-Attention**과 **Transformer** 아키텍처로 발전했다.  
BERT, GPT 등 현재의 대규모 언어 모델은 모두 Transformer 기반이다.

---

### 다음 챕터 예고
- **Chapter 06 실습 1**: 감성 분석 (ex01_sentiment_analysis.ipynb)  
- **Chapter 06 실습 2**: 한국어 텍스트 분류 (ex02_korean_text_classification.ipynb)